In [17]:
import torch
import torch.nn as nn

In [18]:
device = torch.device("cuda:0")

In [19]:
import numpy as np
from torchvision.transforms import Compose
from Utilities.transforms import minmax
from Utilities.dataloader import EEGDataLoader

transforms = Compose([
    minmax,
])

dl = EEGDataLoader("./dataset", transforms)

signal, label = dl.__getitem__(12)
print(np.array(signal).shape)

(16, 8192)


In [94]:
class EEGModel(nn.Module):
    channels = 16
    
    def __init__(self):
        super(EEGModel, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv1d(self.channels, 32, 11),
            nn.ReLU(),
            nn.AvgPool1d(3),
            nn.BatchNorm1d(32),
            nn.Dropout2d(p=0.6),
            
            nn.Conv1d(32, 64, 5),
            nn.ReLU(),
            nn.AvgPool1d(3),
            nn.BatchNorm1d(64),
            nn.Dropout2d(p=0.6),
            
            nn.Conv1d(64, 128, 3),
            nn.ReLU(),
            nn.AvgPool1d(3),
            nn.BatchNorm1d(128),
            nn.Dropout2d(p=0.6),
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(301, 128),
            nn.Linear(128, 64),
            nn.Linear(64, 32),
            nn.Dropout2d(p=0.6),
            nn.Linear(32, 10)
        )
        
        self.fc = nn.Linear(2560, 10)

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        
        x = torch.flatten(x)
        x = self.fc(x)
        
        print(x.shape)
        
        return x

In [95]:
model = EEGModel()
model.to(device)

EEGModel(
  (features): Sequential(
    (0): Conv1d(16, 32, kernel_size=(11,), stride=(1,))
    (1): ReLU()
    (2): AvgPool1d(kernel_size=(3,), stride=(3,), padding=(0,))
    (3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Dropout2d(p=0.6, inplace=False)
    (5): Conv1d(32, 64, kernel_size=(5,), stride=(1,))
    (6): ReLU()
    (7): AvgPool1d(kernel_size=(3,), stride=(3,), padding=(0,))
    (8): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Dropout2d(p=0.6, inplace=False)
    (10): Conv1d(64, 128, kernel_size=(3,), stride=(1,))
    (11): ReLU()
    (12): AvgPool1d(kernel_size=(3,), stride=(3,), padding=(0,))
    (13): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Dropout2d(p=0.6, inplace=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=301, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=64, bias=True)
    (2

In [96]:
from torchsummary import summary

summary(model, (16, 8192))

torch.Size([10])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [-1, 32, 8182]           5,664
              ReLU-2             [-1, 32, 8182]               0
         AvgPool1d-3             [-1, 32, 2727]               0
       BatchNorm1d-4             [-1, 32, 2727]              64
         Dropout2d-5             [-1, 32, 2727]               0
            Conv1d-6             [-1, 64, 2723]          10,304
              ReLU-7             [-1, 64, 2723]               0
         AvgPool1d-8              [-1, 64, 907]               0
       BatchNorm1d-9              [-1, 64, 907]             128
        Dropout2d-10              [-1, 64, 907]               0
           Conv1d-11             [-1, 128, 905]          24,704
             ReLU-12             [-1, 128, 905]               0
        AvgPool1d-13             [-1, 128, 301]               0
      BatchNorm1d-14  

/home/kacper/Documents/venv/lib/python3.10/site-packages/torch/nn/functional.py:1338: UserWarning: dropout2d: Received a 3D input to dropout2d and assuming that channel-wise 1D dropout behavior is desired - input is interpreted as shape (N, C, L), where C is the channel dim. This behavior will change in a future release to interpret the input as one without a batch dimension, i.e. shape (C, H, W). To maintain the 1D channel-wise dropout behavior, please switch to using dropout1d instead.
  warnings.warn("dropout2d: Received a 3D input to dropout2d and assuming that channel-wise "
